In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer
!pip install datasets==2.4.0

In [2]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from tqdm.notebook import tqdm


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [4]:
from datasets import load_dataset,load_metric,Audio
class CommonVoice(torch.utils.data.Dataset):
    """
    A simple class to wrap LibriSpeech and trim/pad the audio to 30 seconds.
    It will drop the last few seconds of a very small portion of the utterances.
    """
    def __init__(self, device=DEVICE):
        self.dataset =load_dataset("mozilla-foundation/common_voice_7_0", "ur", split="test",use_auth_token=True)
        self.dataset = self.dataset.cast_column("audio", Audio(sampling_rate=16_000))
        self.device = device

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, item):
        audio = self.dataset[item]['audio']['array']
        text = self.dataset[item]['sentence']
        sample_rate = self.dataset[item]['audio']['sampling_rate']
        assert sample_rate == 16000
        audio = torch.from_numpy(whisper.pad_or_trim(audio.flatten())).to(self.device)
        mel = whisper.log_mel_spectrogram(audio)
        
        return (mel, text)

In [5]:
dataset = CommonVoice()
loader = torch.utils.data.DataLoader(dataset, batch_size=16)

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


In [13]:
model = whisper.load_model("small")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|███████████████████████████████████████| 461M/461M [00:46<00:00, 10.4MiB/s]


Model is multilingual and has 240,582,912 parameters.


In [14]:
# predict without timestamps for short-form transcription
options = whisper.DecodingOptions(language="ur", without_timestamps=True)

In [15]:
dataset[0]

(tensor([[-0.6812, -0.6812, -0.6812,  ..., -0.6812, -0.6812, -0.6812],
         [-0.6812, -0.6812, -0.6812,  ..., -0.6812, -0.6812, -0.6812],
         [-0.6812, -0.6812, -0.6812,  ..., -0.6812, -0.6812, -0.6812],
         ...,
         [-0.6812, -0.6812, -0.6812,  ..., -0.6812, -0.6812, -0.6812],
         [-0.6812, -0.6812, -0.6812,  ..., -0.6812, -0.6812, -0.6812],
         [-0.6812, -0.6812, -0.6812,  ..., -0.6812, -0.6812, -0.6812]],
        device='cuda:0'), 'اب نیٹ پہ سن لیتے ہیں۔')

In [16]:
!pip install torchaudio==0.11

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
hypotheses = []
references = []

for mels, texts in tqdm(loader):
    results = model.decode(mels, options)
    hypotheses.extend([result.text for result in results])
    references.extend(texts)

  0%|          | 0/9 [00:00<?, ?it/s]

In [18]:
data = pd.DataFrame(dict(hypothesis=hypotheses, reference=references))
data

,hypothesis,reference
0,اب نٹ پے سن دیتی ہے,اب نیٹ پہ سن لیتے ہیں۔
1,مزدور تپکے کے علاوہ سرکاری اور کاروباری لوگ 9،...,مزدور طبقے کے علاوہ سرکاری اور کاروباری لوگ نو...
2,چنیر مشرب جاتے ہیں,جنرل مشرف جاتے ہیں۔
3,دلچیس امریا ہے,دلچسپ امر یہ ہے۔
4,یہ نہیں کہا جائے گا کہ زیاد کی طرف سے دی گئی ا...,یہ نہیں کہا جائے گا کہ زید کی طرف سے دی گئی ای...
...,...,...
137,چھب معملہ کسی عدالت کے ساملے آر,جب معاملہ کسی عدالت کے سامنے آئے۔
138,اسٹریلیا کی ویزے لا مرون پر پانکسسان کی کبردی ...,اسٹریلیاکے ویزے نہ ملنے پر پاکستان کی کبڈی ورل...
139,باہنی روابر تاستوک کی چدر کو کام کر دیتے ہیں,باہمی روابط تعصب کی شدت کو کم کر دیتے ہیں
140,حاضر بائی جان کا موسمِ گرمہ کا وقت,آذربائیجان کا موسم گرما کا وقت


In [19]:
import jiwer
from whisper.normalizers import BasicTextNormalizer

normalizer = BasicTextNormalizer()

In [20]:
data["hypothesis_clean"] = [normalizer(text) for text in data["hypothesis"]]
data["reference_clean"] = [normalizer(text) for text in data["reference"]]
data

,hypothesis,reference,hypothesis_clean,reference_clean
0,اب نٹ پے سن دیتی ہے,اب نیٹ پہ سن لیتے ہیں۔,اب نٹ پے سن دیتی ہے,اب نیٹ پہ سن لیتے ہیں
1,مزدور تپکے کے علاوہ سرکاری اور کاروباری لوگ 9،...,مزدور طبقے کے علاوہ سرکاری اور کاروباری لوگ نو...,مزدور تپکے کے علاوہ سرکاری اور کاروباری لوگ 9 ...,مزدور طبقے کے علاوہ سرکاری اور کاروباری لوگ نو...
2,چنیر مشرب جاتے ہیں,جنرل مشرف جاتے ہیں۔,چنیر مشرب جاتے ہیں,جنرل مشرف جاتے ہیں
3,دلچیس امریا ہے,دلچسپ امر یہ ہے۔,دلچیس امریا ہے,دلچسپ امر یہ ہے
4,یہ نہیں کہا جائے گا کہ زیاد کی طرف سے دی گئی ا...,یہ نہیں کہا جائے گا کہ زید کی طرف سے دی گئی ای...,یہ نہیں کہا جائے گا کہ زیاد کی طرف سے دی گئی ا...,یہ نہیں کہا جائے گا کہ زید کی طرف سے دی گئی ای...
...,...,...,...,...
137,چھب معملہ کسی عدالت کے ساملے آر,جب معاملہ کسی عدالت کے سامنے آئے۔,چھب معملہ کسی عدالت کے ساملے آر,جب معاملہ کسی عدالت کے سامنے آئے
138,اسٹریلیا کی ویزے لا مرون پر پانکسسان کی کبردی ...,اسٹریلیاکے ویزے نہ ملنے پر پاکستان کی کبڈی ورل...,اسٹریلیا کی ویزے لا مرون پر پانکسسان کی کبردی ...,اسٹریلیاکے ویزے نہ ملنے پر پاکستان کی کبڈی ورل...
139,باہنی روابر تاستوک کی چدر کو کام کر دیتے ہیں,باہمی روابط تعصب کی شدت کو کم کر دیتے ہیں,باہنی روابر تاستوک کی چدر کو کام کر دیتے ہیں,باہمی روابط تعصب کی شدت کو کم کر دیتے ہیں
140,حاضر بائی جان کا موسمِ گرمہ کا وقت,آذربائیجان کا موسم گرما کا وقت,حاضر بائی جان کا موسم گرمہ کا وقت,آذربائیجان کا موسم گرما کا وقت


In [21]:
wer = jiwer.wer(list(data["reference_clean"]), list(data["hypothesis_clean"]))

print(f"WER: {wer * 100:.2f} %")

WER: 46.43 %
